# dataretrieval

USGS [National Water Information System (NWIS)](https://waterdata.usgs.gov/nwis) 数据获取现在已经有 python 包了，详见：[USGS-python/dataretrieval](https://github.com/USGS-python/dataretrieval)。

径流站点信息可在这里查询：

[NWIS](https://waterdata.usgs.gov/nwis)中的Site Information和[Map of all sites with links to all available water data for individual sites](https://maps.waterdata.usgs.gov/mapper/index.html).

下面是简单例子

In [1]:
# first import the functions for downloading data from NWIS
import dataretrieval.nwis as nwis

# specify the USGS site code for which we want data.
site = '03339000'


# get instantaneous values (iv)
df = nwis.get_record(sites=site, service='iv', start='2017-12-31', end='2018-01-01')
df.head()

,00010,00010_cd,site_no,00060,00060_cd,00065,00065_cd,00095,00095_cd,00300,...,00400,00400_cd,51289,51289_cd,63680_ysi,63680_ysi_cd,63680_hach,63680_hach_cd,99133,99133_cd
datetime,,,,,,,,,,,,,,,,,,,,,
2017-12-31 00:00:00-06:00,1.0,A,03339000,140.0,A,2.40,A,831.0,A,15.0,...,8.5,A,0.100,A,6.1,A,3.6,A,4.61,A
2017-12-31 00:15:00-06:00,1.0,A,03339000,138.0,A,2.39,A,831.0,A,15.0,...,8.5,A,NaN,NaN,6.1,A,3.6,A,4.61,A
2017-12-31 00:30:00-06:00,1.0,A,03339000,139.0,A,2.40,A,831.0,A,15.0,...,8.5,A,NaN,NaN,6.2,A,3.4,A,4.61,A
2017-12-31 00:45:00-06:00,1.0,A,03339000,139.0,A,2.39,A,832.0,A,14.9,...,8.5,A,NaN,NaN,6.0,A,3.4,A,4.61,A
2017-12-31 01:00:00-06:00,1.0,A,03339000,139.0,A,2.39,A,832.0,A,14.9,...,8.5,A,0.064,A,6.0,A,3.5,A,4.61,A


In [2]:
# get water quality samples (qwdata)
df2 = nwis.get_record(sites=site, service='qwdata', start='2017-12-31', end='2018-01-01')
df2.head()

,agency_cd,site_no,sample_end_dt,sample_end_tm,tm_datum_rlbty_cd,coll_ent_cd,medium_cd,project_cd,aqfr_cd,tu_id,body_part_id,hyd_cond_cd,samp_type_cd,hyd_event_cd,sample_lab_cm_txt
datetime,,,,,,,,,,,,,,,


In [3]:
# get basic info about the site
df3 = nwis.get_record(sites=site, service='site')
df3.head()

,agency_cd,site_no,station_nm,site_tp_cd,lat_va,long_va,dec_lat_va,dec_long_va,coord_meth_cd,coord_acy_cd,...,local_time_fg,reliability_cd,gw_file_cd,nat_aqfr_cd,aqfr_cd,aqfr_type_cd,well_depth_va,hole_depth_va,depth_src_cd,project_no
0,USGS,03339000,"VERMILION RIVER NEAR DANVILLE, IL",ST,400603,873550,40.10087,-87.597242,M,S,...,N,NaN,NNNNNNNN,NaN,NaN,NaN,NaN,NaN,NaN,100


下面是老方法。

主要参考了博客[径流观测数据](http://mengfeimu.com/2019/03/06/streamflow/)，批量下载：

In [1]:
import pandas as pd
import requests

site_id = '02096846'
year1 = 2011
year2 = 2012
# url中除了site_id外，还有两个参数设置需要注意，一个是parameter，也就是数据类型，这里00060表示径流的编号；另一个就是statistics，即数据的值类型，这里00003表示均值
# 根据官方的Q&A：How do I get help refining my URL query? Please tell us what you want to do by sending an email to gs-w_support_nwisweb@usgs.gov.
# 这里url只给出了径流的编号，统计值的编号暂时不知道如何给出
url = 'https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no={}&\
referred_module=sw&period=&begin_date={}-01-01&end_date={}-12-31'.format(site_id,year1,year2)
print(url)
# pandas从0.19.2及之后的版本都可以直接从url中读取数据，skiprows需要读文件的时候判断
# 下载的数据文件，数字的起始行数不同，不过都是最后一行注释“#”之后第三行
skip_rows_index=list(range(0,28))
print(type(skip_rows_index))
print(skip_rows_index)
skip_rows_index.append(29)
print(skip_rows_index)
df_flow = pd.read_csv(url, skiprows=skip_rows_index, sep='\t', dtype={'site_no': str})
print(df_flow[0:5])

# 原数据的列名并不好用，这里修改
columns_names=df_flow.columns.tolist()
for column_name in columns_names:
    if '_00060_00003' in column_name and '_00060_00003_cd' not in column_name:
        df_flow.rename(columns={column_name:'flow'},inplace=True)
    elif '_00060_00003_cd' in column_name:
        df_flow.rename(columns={column_name:'mode'},inplace=True)

columns=['agency_cd','site_no','datetime','flow','mode']
df_flow=df_flow.loc[:,columns]
print(df_flow[0:5])

df_flow.index = pd.DatetimeIndex(df_flow.datetime)
# 如果需要转换单位：convert unit: f3/s --> m3/s
df_flow.flow *= 0.0283168
if len(df_flow.index) == len(pd.date_range('{}0101'.format(year1), '{}1231'.format(year2),freq='D')):
    print('Good! No missing flow data!')
else:
    print('Opoo! Please check missing data!')
    

# 通过直接读取网页的方式批量获取数据，然后存入txt文件
site_id = '05594100,01013500'
url = 'https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no={}&\
referred_module=sw&period=&begin_date={}-01-01&end_date={}-12-31'.format(site_id,year1,year2)
r = requests.get(url)
with open('usgs-test.txt','w') as f:
    f.write(r.text)
print("写入成功！")

# 批量读取数据的方式更快，不过要把数据组织为每个站点的单独页面，需要进行额外处理：用url读取/写入文件后即刻处理。
# 这种在数据量较大时有可能过于耗内存，因此最好是利用爬虫相关机制进行处理，这里暂时这样处理。
# 首先读取第16行，判断总共有多少个sites的数据被写入


https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no=02096846&referred_module=sw&period=&begin_date=2011-01-01&end_date=2012-12-31
<class 'list'>
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29]
  agency_cd   site_no    datetime  86273_00060_00001 86273_00060_00001_cd  \
0      USGS  02096846  2011-01-01               1.00                    A   
1      USGS  02096846  2011-01-02               1.68                    A   
2      USGS  02096846  2011-01-03               3.63                    A   
3      USGS  02096846  2011-01-04               3.16                    A   
4      USGS  02096846  2011-01-05               2.27                    A   

   86274_00060_00002 86274_00060_00002_cd  86275_00060_00003  \
0               0.65                    A               0.78   
1               0.87                    